In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np
import time

In [ ]:
API = "RGAPI-144650db-9573-4a50-82ec-d289ff77b9d8"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API
}

df = pd.read_csv("account.csv")
user_account = df.account

In [ ]:
URL = "https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}".format(user_account[0])
res = req.get(URL, headers=headers)
res.json().keys()

In [ ]:
match_id_list = []  # match id를 저장해둘 리스트

## 모든 account를 돌면서 매치 id들을 가져옴
start_time = time.time()
tally = 1
for account in user_account:
    URL = "https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}".format(account)
    res = req.get(URL, headers=headers)
    
    # API 사용량 제한에 따라서 생기는 오류 처리
    if res.status_code == 200:
        pass
    elif res.status_code == 429:
        while res.status_code != 200:
            print("API 사용량 초과. 30초 대기")
            time.sleep(30)
            res = req.get(URL, headers=headers)
            
    elif res.status_code == 503:
        while res.status_code != 200:
            print("알 수 없는 오류")
            time.sleep(10)
            res = req.get(URL, headers=headers)
            
    elif res.status_code == 403:
        while res.status_code != 200:
            print("API 갱신 필요")
            break
    
    
    # 매치 ID 저장
    for i in range(len(res.json()["matches"])):
        match_id = res.json()["matches"][i]["gameId"]
        # 중복된 매치 ID 빼고 저장
        if match_id not in match_id_list:
            match_id_list.append(match_id)
            
    # 현재까지 작업량을 출력
    if len(match_id_list) > (3000*tally):
        print("현재까지 가져온 match id: {}".format(len(match_id_list)))
        print("소요된 시간: {}".format(time.time()-start_time))
        print()
        tally += 1

        
print("총 걸린 시간: {}".format(time.time()-start_time))

In [ ]:
len(match_id_list)

In [ ]:
pd.DataFrame(match_id_list).to_csv("match_id.csv", index=False)